## Repo statistics for trydotnet

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

Attempting to install package Octokit, version 0.32.0.........................................................done!

Successfully added reference to package Octokit, version 0.32.0

Attempting to install package NodaTime, version 2.4.6...............done!

Successfully added reference to package NodaTime, version 2.4.6

In [2]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

In [3]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

In [27]:
var createdIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                Filter = IssueFilter.Created
            };
var closedIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                State = ItemStateFilter.Closed
            };
var thisYearIssuesRequest = new RepositoryIssueRequest
            {
                State = ItemStateFilter.All,
                Since = startOfTheYear.ToDateTimeUnspecified()
            };

In [28]:
var createdThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", createdIssuesRequest);
var closedThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", closedIssuesRequest);
var thisYearIssues = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", thisYearIssuesRequest);

In [49]:
var openSoFar = createdThisMonth.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "open");
var openByMonth = openSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()});
var closedSoFar = thisYearIssues.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "closed");
var closedByMonth = closedSoFar.GroupBy(i => new { i.ClosedAt.Value.Year, i.ClosedAt.Value.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()});

In [45]:
var lineChart = Chart.Line(createdThisMonth.GroupBy(i => new DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day)).Select(g => new Tuple<DateTime,int>(g.Key, g.Count())));
lineChart.WithTitle("Issue Open By Day");
display(lineChart);

In [46]:
display(openSoFar.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}));

index,CreatedAt,Title,State,Number
0,2019-08-19 08:05:46Z,"Provide documentation for using wasm ""mode""",open,382
1,2019-08-21 21:15:50Z,"""args"" in Main(string[] args) is null",open,390
2,2019-08-22 20:09:04Z,Adding extensions to the .NET kernel using nuget packages,open,395
3,2019-08-29 16:08:30Z,Try the .NET Kernel,open,408
4,2019-09-03 16:30:59Z,"After running a %%html / %%markdown / %%javascript, ""running"" indicator is not reset",open,423
5,2019-09-05 23:22:12Z,Publish interactive packages,open,427
6,2019-09-06 01:29:27Z,Creating kernel extensions that are not part of other nuget packages,open,428
7,2019-09-07 09:56:08Z,"Android on screen function is not working, back space can't be used to edit",open,429
8,2019-09-10 21:09:41Z,Kernel Info reply doesn't reflect the kernel language and the editor coloring is not correct,open,437
9,2019-09-15 20:26:02Z,System.ArgumentException: An item with the same key has already been added. Key: .wasm,open,445


In [47]:
var lineChart = Chart.Line(openByMonth.Select(g => new Tuple<DateTime, int>(new DateTime(g.Date.Year, g.Date.Month, 1),g.Count)));
lineChart.WithTitle("Issue Open By Month");
display(lineChart);

In [54]:
var openDataPoints = openByMonth
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();


var closedDataPoints = closedByMonth
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openSeries = new Graph.Scatter
{
    name = "Open",
    y = openDataPoints.Select(g => g.y ).ToArray(),
    x = openDataPoints.Select(g => g.x ).ToArray()
};

var closeSeries = new Graph.Scatter
{
    name = "Close",
    y = closedDataPoints.Select(g => g.y ).ToArray(),
    x = closedDataPoints.Select(g => g.x ).ToArray()
};

var chart = Chart.Plot(new[] {openSeries, closeSeries});
chart.WithTitle("Open vs Close");
display(chart);